<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mise-en-place-de-l'environnement" data-toc-modified-id="Mise-en-place-de-l'environnement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mise en place de l'environnement</a></span><ul class="toc-item"><li><span><a href="#Test-de-l'Environnement" data-toc-modified-id="Test-de-l'Environnement-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Test de l'Environnement</a></span></li></ul></li><li><span><a href="#Implémentation-de-l'Algorithme-Deep-Deterministic-Policy-Gradient-(DDPG)" data-toc-modified-id="Implémentation-de-l'Algorithme-Deep-Deterministic-Policy-Gradient-(DDPG)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implémentation de l'Algorithme Deep Deterministic Policy Gradient (DDPG)</a></span><ul class="toc-item"><li><span><a href="#Création-de-l'Acteur" data-toc-modified-id="Création-de-l'Acteur-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Création de l'Acteur</a></span></li><li><span><a href="#Création-du-Critique" data-toc-modified-id="Création-du-Critique-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Création du Critique</a></span></li><li><span><a href="#Création-du-Générateur-de-Bruit" data-toc-modified-id="Création-du-Générateur-de-Bruit-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Création du Générateur de Bruit</a></span></li><li><span><a href="#Gestion-de-l'Experience-Replay" data-toc-modified-id="Gestion-de-l'Experience-Replay-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Gestion de l'Experience Replay</a></span></li><li><span><a href="#Mise-à-jour-des-réseaux-cibles" data-toc-modified-id="Mise-à-jour-des-réseaux-cibles-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Mise à jour des réseaux cibles</a></span></li><li><span><a href="#Apprentissage" data-toc-modified-id="Apprentissage-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Apprentissage</a></span></li><li><span><a href="#Diagnostique" data-toc-modified-id="Diagnostique-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Diagnostique</a></span></li><li><span><a href="#Réglage-des-paramètres-d'apprentissage" data-toc-modified-id="Réglage-des-paramètres-d'apprentissage-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Réglage des paramètres d'apprentissage</a></span></li></ul></li></ul></div>

Les modèles (physiques ou corrélatifs) des pneumatiques jouent un rôle essentiel dans la mise au points de scénarios de conception ainsi que dans l'évaluation des performances des nouvelles gammes ou de gammes existantes. Ainsi, le modèle de rigidité de dérive vu dans le TP portant sur l'Optimisation Bayesienne, peut être exploité à travers des chaines de simulation pour juger de la qualité de pneumatiques en terme de critère de comportement, d'adhérence, d'endurance ou encore de temps au tour. C'est à cette performance que nous allons nous intéresser ici.

Plus précisément, l'exercice consiste à mettre en place un environnement de simulation basé sur de l'apprentissage par renforcement qui a pour objectif de trouver les controles optimaux à appliquer à un véhicule pour que ce dernier puisse parcourir un circuit circulaire avec la vitesse la plus élevée possible.

## Mise en place de l'environnement

En l'occurence, les états que l'on va considérer pour notre environnement sont:
- $x$: position du véhicule selon la direction $\vec{X}$
- $y$: position du véhicule selon la direction $\vec{Y}$
- $\dot{x}$: vitesse du véhicule selon la direction $\vec{X}$
- $\dot{y}$: vitesse du véhicule selon la direction $\vec{Y}$
- $\psi$: l'angle de lacet du véhicule
- $\dot{\psi}$: vitesse de lacet du véhicule



Les actions qui seront utilisées sont:
- $v$: la vitesse
- $\alpha$: l'angle de braquage

L'environnement que l'on va exploiter s'appuie sur le package Gym de la société OpenAI (https://gym.openai.com/). Un tel environnement s'appuie sur l'utilisation d'objets héritant de la classe *gym.Env* et comportant les méthodes suivantes:
- **__init__**: constructeur définissant les expaces d'actions (*action_space*) et d'observations (*observation_space*)
- **reset**: méthode permettant de réinitialiser les états
- **step**: fonction qui prend en entrée les valeurs des actions et renvoie les nouveaux états de l'environnement, le reward ainsi qu'un booléen indiquant s'il est nécessaire de réinitialiser les états 
- **render**: méthode qui affiche l'état de l'environnement et différentes informations le concernant 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
from gym_gmmcar.envs.circle_env import CircleEnv

Notre objectif étant de rester sur le cirucuit tout en allant le plus vite possible, quel(-s) reward(-s) peut-on envisager? Implémenter l'un d'entre eux en complétant la méthode *get_reward* de la classe *OttEnv* ci-dessous:

In [64]:
class OttEnv(CircleEnv):
    """
    Environnement de simulation pour une voiture de course suivant une trajectoire circulaire aussi vite que possible
    """

    def __init__(
            self,
            target_velocity=1.0,
            radius=1.0,
            dt=0.035,
            model_type='BrushTireModel',
            robot_type='RCCar',
            mu_s=1.37,
            mu_k=1.96,
            eps=0.05
    ):

        super().__init__(
            target_velocity=target_velocity,
            radius=radius,
            dt=dt,
            model_type=model_type,
            robot_type=robot_type,
            mu_s=mu_s,
            mu_k=mu_k
        )

        self.eps = eps


    def get_reward(self, state, action):
        """
        Définition de la fonction de Reward
        """
        r = self.radius
        v = self.target_velocity
        x, y, _, x_dot, y_dot, _ = state
        vitesse = np.sqrt(x_dot**2 + y_dot**2)
        distance = np.sqrt(x**2 + y**2)

        # Reward à définir
        distance_penalty = - ((distance - r) / r)**2  
        vitesse_penalty = - ((vitesse - v) / v)**2
        reward = 1 + distance_penalty + vitesse_penalty
        
        info = {}
        info['dist'] = distance
        info['vel'] = vitesse
        return reward, info

### Test de l'Environnement

Tester l'environnement en considérant un épisode de 100 pas de temps et des actions aléatoires et/ou fixes. Pour ce faire, compléter le script ci-dessous en définissant les actions à appliquer à chaque pas.

In [3]:
env = OttEnv()
obs = env.reset()
env.render()

episode = 1
for step in range(100):
    #action = env.action_space.sample()
    action = np.array([100,1])
    new_state, reward, done, info = env.step(action)
    #print(new_state)
    env.render()

C:\Users\yoyoc\anaconda3\envs\michelin\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


[-1.29091095 -0.02173952  4.00127525  1.42926365 -0.46067099  2.5398193 ]
[-1.33575665 -0.05475184  4.1093056   1.6334305  -0.42530022  3.61851293]
[-1.37990306 -0.0990019   4.25358022  1.82524902 -0.47998891  4.61457178]
[-1.42183267 -0.15432812  4.43180592  1.98472041 -0.62122258  5.56561953]
[-1.45932512 -0.22016624  4.64302938  2.08426745 -0.840745    6.50344375]
[-1.48975915 -0.29521034  4.88698598  2.09299329 -1.12425497  7.43532153]
[-1.51060519 -0.37720109  5.16327301  1.9811809  -1.4463198   8.34718448]
[-1.5199865  -0.46295692  5.47079625  1.72727307 -1.76568389  9.2181665 ]
[-1.51713752 -0.54859948  5.80811668  1.32559791 -2.02303392 10.05279357]
[-1.50273813 -0.62994153  6.17430884  0.79671376 -2.15020311 10.87183817]
[-1.4790865  -0.70312421  6.56924154  0.19489658 -2.0837941  11.69908504]
[-1.44968733 -0.76546318  6.9919195  -0.35947731 -1.83827609 12.20697104]
[-1.41205094 -0.8175392   7.40155678 -0.66004407 -1.68161973 11.11376237]
[-1.36532189 -0.85731276  7.76913183 -

In [32]:
state = env.reset()
print(f"Initial state shape: {state.shape}")  # etat initial 

next_state, reward, done, info = env.step(action)
print(f"Next state shape: {next_state.shape}")  # etat suivant


Initial state shape: (4,)
Next state shape: (6,)


Etant donné les caractéristiques du problème considéré, quel type de méthode devrait-on appliquer?

## Implémentation de l'Algorithme Deep Deterministic Policy Gradient (DDPG)

## 

Pour tenter de trouver les commandes optimales à appliquer, nous allons ici utiliser une approche DDPG. Pour ce faire, la première étape à réaliser est d'implémenter cette méthode en s'appuyant sur le pseudo-code suivant vu en cours:
![DDPG.png](DDPG.png "Algorithme DDPG")


/!\ bien faire step by step, revoir le **cours**, acteur critique, DQL actions discrètes => problème car espace d'actions continu; pour résourdre ce problème => acteur critique + DQL ~= DDPG

### Création de l'Acteur

Pour rappel, l'acteur a pour objectif d'estimer un politique $\mu(s_{t})$. Dans un premier temps, créer un acteur à partir d'une fonction ou d'une classe en définissant un modèle neuronal tensorflow ayant l'architecture suivante:
- une première couche cachée dense comportant 256 neurones et une fonction d'activation de type RELU
- une seconde couche cachée dense comportant 256 neurones et une fonction d'activation de type RELU
- une couche de sortie dense comportant un nombre de neurones égal au nombre d'actions et une fonction d'activation de type tanh

<ins>**Remarque:**</ins> Les sorties étant bornées entre -1 et 1, ne pas oublier de dénormaliser pour générer des valeurs d'actions conformes à l'espace des actions.
<ins>**Conseil:**</ins> Pour pouvoir tester différentes architectures par la suite, paramétrer les couches à l'aide d'une variable indiquant le nombre de neurones. 

In [14]:
def create_actor(num_actions, action_bounds, num_neurons=256, state_dim=4):
    inputs = layers.Input(shape=(state_dim,), name="state")
    x = layers.Dense(num_neurons, activation='relu')(inputs)
    x = layers.Dense(num_neurons, activation='relu')(x)
    normalized_actions = layers.Dense(num_actions, activation='tanh')(x)

    min_action, max_action = action_bounds # on dénomalise l'output
    scaled_actions = layers.Lambda(
        lambda x: min_action + (x + 1.0) * (max_action - min_action) / 2.0,
        name="scaled_actions"
    )(normalized_actions)

    model = tf.keras.Model(inputs, scaled_actions, name="actor_model")
    return model

### Création du Critique

Pour rappel, le critique a pour objectif d'estimer la valeur $Q(s_{t},a_{t})$Dans un premier temps, créer un critique à partir d'une fonction ou d'une classe en définissant un modèle neuronal tensorflow de la manière suivante:
- Créer un réseau prenant en entrée les états avec:
  - une première couche cachée dense comportant 16 neurones et une fonction d'activation de type RELU
  - une seconde couche cachée dense comportant 32 neurones et une fonction d'activation de type RELU
- Créer un réseau prenant en entrée les actions avec une couche cachée dense comportant 32 neurones et une fonction d'activation de type RELU
- Concaténer les sorties des 2 réseaux précédents via la méthode "*Concatenate*"
- Créer un réseau prenant les entrées la concaténation des tenseurs précédents avec:
  - une première couche cachée dense comportant 256 neurones et une fonction d'activation de type RELU
  - une seconde couche cachée dense comportant 256 neurones et une fonction d'activation de type RELU
  - une couche de sortie dense comportant 1 neurone sans fonction d'activation 

<ins>**Conseil:**</ins> Pour pouvoir tester différentes architectures par la suite, paramétrer les couches à l'aide d'une variable indiquant le nombre de neurones. 

In [15]:
def create_critic(state_dim, action_dim, num_neurons_state=16, num_neurons_action=32, num_neurons_fc=256):
    # etats
    state_input = layers.Input(shape=(state_dim,), name="state")
    x_state = layers.Dense(num_neurons_state, activation='relu')(state_input)
    x_state = layers.Dense(num_neurons_state * 2, activation='relu')(x_state)

    # actions
    action_input = layers.Input(shape=(action_dim,), name="action")
    x_action = layers.Dense(num_neurons_action, activation='relu')(action_input)

    # actions  + etats
    concatenated = layers.Concatenate()([x_state, x_action])
    x = layers.Dense(num_neurons_fc, activation='relu')(concatenated)
    x = layers.Dense(num_neurons_fc, activation='relu')(x)
    q_value = layers.Dense(1)(x)


    model = tf.keras.Model(inputs=[state_input, action_input], outputs=q_value, name="critic_model")
    return model

### Création du Générateur de Bruit

Comme précisé en cours, l'approche DDPG génère les actions de manière déterministe, ce qui engendre mécaniquement une démarche purement basée sur de l'exploitation. Pour éviter d'être coincé dans un optimum local, il est nécessaire d'appliquer une stratégie d'exploration. En l'occurrence, cette exploration est gérée via l'ajout d'un bruit à l'action générée par l'acteur.
Ce bruit est généré via un processus stochastique de type ***Ornstein-Uhlenbeck*** défini par l'équation différentielle stochastique:

$dx_{t}=\theta(\nu-x_{t})dt+\sigma\sqrt{d_{t}}u$ avec $u\sim \mathcal{N}(0,1)$

Créer une fonction ou classe permettant de générer ce bruit avec $\theta=0.15$ et $d_{t}=1e-2$.

In [16]:
# Define the noise generator with proper initialization of the state variable
def bruit(action_dim, mu=0.0, theta=0.15, sigma=0.2, dt=1e-2):
    state = np.zeros(action_dim)  # Initialize the state variable
    while True:
        dx = theta * (mu - state) * dt + sigma * np.sqrt(dt) * np.random.randn(action_dim)
        state = state + dx
        yield state

### Gestion de l'Experience Replay

Afin de ne pas oublier les expériences passées et réduire les corrélations entre expériences, un tirage aléatoire de $N$ tuples (état présent, action, reward, état suivant) stockés dans un buffer de taille $B$.
Créer une fonction ou classe permettant de:
- Initialiser un buffer de taille $B$ à 0
- Sauvegarder à chaque pas de temps un 4-uplet (état présent, action, reward, état suivant)
- Tirer aléatoirement $N$ tuples (état présent, action, reward, état suivant)

In [43]:
def Buffer_Init(buffer_size, state_dim, action_dim):
    buffer = {
        'states': np.zeros((buffer_size, state_dim), dtype=np.float32),
        'actions': np.zeros((buffer_size, action_dim), dtype=np.float32),
        'rewards': np.zeros((buffer_size, 1), dtype=np.float32),
        'next_states': np.zeros((buffer_size, state_dim), dtype=np.float32),
        'indices': 0}
    return buffer

def save(buffer, state, action, reward, next_state):
    index = buffer['indices']
    buffer['states'][index] = state
    buffer['actions'][index] = action
    buffer['rewards'][index] = reward
    buffer['next_states'][index] = next_state
    buffer['indices'] = (index + 1) % buffer['states'].shape[0]

def random_pick(buffer, batch_size):
    indices = np.random.choice(buffer['states'].shape[0], batch_size, replace=False)
    states = buffer['states'][indices]
    actions = buffer['actions'][indices]
    rewards = buffer['rewards'][indices]
    next_states = buffer['next_states'][indices]
    return states, actions, rewards, next_states

### Mise à jour des réseaux cibles

Comme présenté en cours, la gestion des cibles mouvantes se fait via la mise en place de réseaux cibles. En l'occurrence, deux réseaux cibles sont utilisés: l'un pour l'acteur et l'autre pour le critique.
Créer une fonction ou classe qui mette à jour les poids des réseaux cibles.

In [18]:
# Mise à jour des réseaux cibles

def update_target_networks(main_network, target_network, tau):
    main_weights = main_network.get_weights() 
    target_weights = target_network.get_weights()

    new_target_weights = [tau * main_weight + (1 - tau) * target_weight for main_weight, target_weight in zip(main_weights, target_weights)] # pas sur sur

    target_network.set_weights(new_target_weights)

### Apprentissage

Utiliser l'ensembles des fonctions précédemment construites pour implémenter l'apprentissage présenté par le pseudo-code apparaissant plus haut avec les paramètres suivants:
- learning rate de l'acteur:0.002
- learning rate du critique: 0.001
- paramètre du générateur de bruit $\sigma$: 0.2
- paramètre du générateur de bruit $\nu$: 0
- nombre totale d'épisode $M$: 100
- facteur d'escompte $\gamma$: 0.99
- paramètre mise à jour des réseaux cible $\tau$: 0.005
- taille du buffer $B$: 1000
- taille $N$ des batchs: 100

Pour pouvoir mener un diagnosqtique de l'apprentissage, stocker les rewards cumulés à la fin de chaque épisode dans une liste.

In [57]:
GAMMA = 0.99
TAU = 0.005
ACTOR_LR = 0.001
CRITIC_LR = 0.001
BUFFER_SIZE = 1000
BATCH_SIZE = 100
EPISODES = 100
MAX_STEPS = 200


def train_step(buffer, actor, critic, target_actor, target_critic, actor_optimizer, critic_optimizer):
    states, actions, rewards, next_states = random_pick(buffer, BATCH_SIZE)

    next_actions = target_actor(tf.convert_to_tensor(next_states, dtype=tf.float32))
    next_q_values = target_critic([tf.convert_to_tensor(next_states, dtype=tf.float32), next_actions])
    target_q_values = rewards + GAMMA * next_q_values * (1.0 - np.array([done for done in rewards]))

    #maj du critique 
    with tf.GradientTape() as tape:
        q_values = critic([tf.convert_to_tensor(states, dtype=tf.float32), tf.convert_to_tensor(actions, dtype=tf.float32)])
        critic_loss = tf.reduce_mean(tf.square(target_q_values - q_values))
    critic_grads = tape.gradient(critic_loss, critic.trainable_variables)
    critic_optimizer.apply_gradients(zip(critic_grads, critic.trainable_variables))

    # maj de l'acteur 
    with tf.GradientTape() as tape:
        actions = actor(tf.convert_to_tensor(states, dtype=tf.float32))
        actor_loss = -tf.reduce_mean(critic([tf.convert_to_tensor(states, dtype=tf.float32), actions]))
    actor_grads = tape.gradient(actor_loss, actor.trainable_variables)
    actor_optimizer.apply_gradients(zip(actor_grads, actor.trainable_variables))




def train_ddpg(env):
    #init des dimensions et intervalles
    state_dim = 6 #env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    action_bounds = (env.action_space.low, env.action_space.high)
    #print(state_dim,action_dim,action_bounds)

    #init  reseaux
    actor = create_actor(action_dim, action_bounds, state_dim=state_dim)
    critic = create_critic(state_dim, action_dim)
    target_actor = create_actor(action_dim, action_bounds, state_dim=state_dim)
    target_critic = create_critic(state_dim, action_dim)

    target_actor.set_weights(actor.get_weights())
    target_critic.set_weights(critic.get_weights())

    # optim
    actor_optimizer = tf.keras.optimizers.Adam(ACTOR_LR)
    critic_optimizer = tf.keras.optimizers.Adam(CRITIC_LR)

    #init buffer
    buffer = Buffer_Init(BUFFER_SIZE, state_dim, action_dim)

    #bruit pour exploration 
    noise_gen = bruit(action_dim)

    # Boucle d'apprentissage
    for episode in range(EPISODES):
        env.reset()
        state = env._state
        episode_reward = 0

        for step in range(MAX_STEPS):
            state_tensor = tf.expand_dims(tf.convert_to_tensor(state, dtype=tf.float32), axis=0)
            action = actor(state_tensor).numpy()[0] + next(noise_gen)

            next_state, reward, done, _ = env.step(action)
            #print(next_state)
            save(buffer, state, action, reward, next_state)

            if buffer['indices'] >= BATCH_SIZE:
                train_step(buffer, actor, critic, target_actor, target_critic, actor_optimizer, critic_optimizer)

            state = next_state
            episode_reward += reward

            if done:
                break

        # maj des paramètres
        update_target_networks(actor, target_actor, TAU)
        update_target_networks(critic, target_critic, TAU)

        print(f"Épisode {episode + 1}, Reward total = {episode_reward}")

In [65]:
env = OttEnv()
train_ddpg(env)

Épisode 1, Reward total = -3615.7797544156438
Épisode 2, Reward total = -4.864676471436246
Épisode 3, Reward total = 9.093496261752504
Épisode 4, Reward total = 35.780727284235056
Épisode 5, Reward total = 17.06023856648893
Épisode 6, Reward total = 21.535845948119842
Épisode 7, Reward total = 37.33985626136411
Épisode 8, Reward total = 25.02907288401775
Épisode 9, Reward total = 8.060994184200867
Épisode 10, Reward total = 16.610842398396535
Épisode 11, Reward total = 19.93170731563263
Épisode 12, Reward total = 31.954791027001228
Épisode 13, Reward total = 23.75888623958979
Épisode 14, Reward total = 2.6533494010540797
Épisode 15, Reward total = 4.217466843908708
Épisode 16, Reward total = 9.395423095253555
Épisode 17, Reward total = 33.56006114006121
Épisode 18, Reward total = -9.589848938906664
Épisode 19, Reward total = 12.722309712390096
Épisode 20, Reward total = 25.8498951155992
Épisode 21, Reward total = 6.892770732760539
Épisode 22, Reward total = 22.211933443407982
Épisode 2

In [49]:
state = env._state
state

array([-1.21152265, -0.06062286,  4.9144126 ,  1.90897533, -0.14733457,
       -0.55039337])

### Diagnostique

Afficher l'évolution de la moyenne des rewards cumulés calculée tous les 20 épisodes.

In [70]:
# Affichage de la moyenne des rewards cumulés


### Réglage des paramètres d'apprentissage

Essayer différents paramètres utilisés lors de l'apprentissage ainsi que différentes architecture de réseaux de neurones. Comment pourrait-on automatiser une recherche intelligente de cesdivers paramètres?

In [12]:
# Tests et évaluation avec différentes configurations

